In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time

# Load dataset
df = pd.read_excel("ENB2012_data.xlsx")

# Define features and targets
X = df.drop(columns=['Y1', 'Y2'])  
y = df[['Y1', 'Y2']]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.1, random_state=42)

# Train the model once and measure training time
start_time = time.time()
xgb_model.fit(X_train, y_train)
training_time = time.time() - start_time

# Make predictions using the trained model
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate performance for both Y1 (Heating) and Y2 (Cooling)
mae_y1_xgb = mean_absolute_error(y_test.iloc[:, 0], y_pred_xgb[:, 0])
mse_y1_xgb = mean_squared_error(y_test.iloc[:, 0], y_pred_xgb[:, 0])
r2_y1_xgb = r2_score(y_test.iloc[:, 0], y_pred_xgb[:, 0])

mae_y2_xgb = mean_absolute_error(y_test.iloc[:, 1], y_pred_xgb[:, 1])
mse_y2_xgb = mean_squared_error(y_test.iloc[:, 1], y_pred_xgb[:, 1])
r2_y2_xgb = r2_score(y_test.iloc[:, 1], y_pred_xgb[:, 1])

# Print results
print("XGBoost Regressor Results:")
print(f"Heating Load (Y1) - MAE: {mae_y1_xgb}, MSE: {mse_y1_xgb}, R2: {r2_y1_xgb}")
print(f"Cooling Load (Y2) - MAE: {mae_y2_xgb}, MSE: {mse_y2_xgb}, R2: {r2_y2_xgb}")
print(f"Training Time: {training_time:.2f} seconds")

XGBoost Regressor Results:
Heating Load (Y1) - MAE: 0.26015774352829174, MSE: 0.16348012315575197, R2: 0.9984315806927794
Cooling Load (Y2) - MAE: 0.5237115686590023, MSE: 0.7954066867255357, R2: 0.9914156010506407
Training Time: 0.32 seconds
